In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import itertools as it

from latticeproteins.thermodynamics import LatticeThermodynamics
from latticeproteins.interactions import miyazawa_jernigan
from latticeproteins.conformations import Conformations, ConformationList
from latticeproteins.sequences import find_differences, _residues

In [2]:
class PredictedLattice(object):
    """Lattice model predictor using epistasis."""
    def __init__(self, wildtype, temp, confs, target=None):
        self.wildtype = wildtype
        self.temp = temp
        self.conformations = confs
        self.target = target
        self._lattice = LatticeThermodynamics(self.temp, self.conformations)

        combos = []
        sites = list(range(self.conformations.length()))
        self.dG0 = round(self._lattice.stability(self.wildtype, target=self.target), 9)

        # Calculate first order coefs
        self.dGs = {}
        for i in sites:
            other_sites = sites[:]
            other_sites.remove(i)
            for aa in _residues:
                combos.append((i, aa))

        for c in combos:
            seq = list(self.wildtype[:])
            seq[c[0]] = c[1]
            # Calculate dG as dG_wt -
            self.dGs[c] = round(self._lattice.stability(seq, target=self.target), 9) - self.dG0

        # Calculate second order coefs
        combos = []
        sites = list(range(self.conformations.length()))
        for i in sites:
            other_sites = sites[:]
            other_sites.remove(i)
            for aa in _residues:
                for j in other_sites:
                    for aa2 in _residues:
                        combos.append((i,aa,j,aa2))

        for c in combos:
            seq = list(self.wildtype[:])
            seq[c[0]] = c[1]
            seq[c[2]] = c[3]
            # Calculate dG2
            self.dGs[c] = round(self._lattice.stability(seq, target=self.target), 9) - (self.dG0 + self.dGs[(c[0],c[1])]+ self.dGs[(c[2],c[3])])

    def stability(self, seq, target=None):
        # Get additive coefs to build predictions
        if target != self.target:
            raise Exception("Target does not match wildtype target.")
        loci = find_differences(self.wildtype, seq)
        add = [(pair[0], seq[pair[0]]) for pair in it.combinations(loci, 1)]
        pairs = [(pair[0], seq[pair[0]], pair[1], seq[pair[1]]) for pair in it.combinations(loci, 2)]
        dgs = add + pairs
        stability = float(self.dG0)
        for coef in dgs:
            stability += self.dGs[coef]
        return stability


def kmax2d(arr, k):
    n, m = arr.shape
    vec = arr.flatten()
    vec_ = vec.argsort()[::-1]
    top_vec = vec_[:k]
    row = top_vec // n
    col = top_vec % n
    return row, col

In [13]:
def fracfolded(dG):
    return 1.0 / (1.0 + np.exp(dG))

def fixation(fitness1, fitness2, N=10e8, *args, **kwargs):
    """ Simple fixation probability between two organism with fitnesses 1 and 2.
    Note that N is the effective population size.
    .. math::
        p_{\\text{fixation}} = \\frac{1 - e^{-N \\frac{f_2-f_1}{f1}}}{1 - e^{-\\frac{f_2-f_1}{f1}}}
    """
    sij = (fitness2 - fitness1)/abs(fitness1)
    # Check the value of denominator
    denominator = 1 - np.exp(-N * sij)
    numerator = 1 - np.exp(- sij)
    # Calculate the fixation probability
    fixation = numerator / denominator
    fixation = np.nan_to_num(fixation) 
    return fixation

def walk(seq, lattice, pdensity=0.95, target=None):
    """"""
    # We create a matrix of possible mutations. Each row is site in the protein
    # and each column is one of the 20 possible amino acids. We calculate the
    # stability of each mutation in this matrix and choose the n_top_mutations.
    # Each of these new mutations become the new starting genotypes for the next
    # move. We repeat this process until all paths reach a dG max.
    
    dG0 = round(lattice.stability(seq, target=target), 9)
    ff0 = fracfolded(dG0)
    unfinished_paths = [[seq]]
    unfinished_ff = [[ff0]]
    unfinished_fix = [[0]]
    finished_paths = []
    finished_ff = []
    finished_fix = []
    
    m = 0
    
    while len(unfinished_paths) != 0:
        finished = True
        updated_unfinished_paths = []
        updated_unfinished_ff = []
        updated_unfinished_fix = []
        updated_finished_paths = []
        updated_finished_ff = []
        updated_finished_fix = []
        
        # Iterate through paths
        for l, path in enumerate(unfinished_paths):
            
            # construct new trajectories
            path_ff = unfinished_ff[l]
            path_fix = unfinished_fix[l]
            # We're grow our set of paths by n_top_mutations.
            mutant = list(path[-1])
            current_ff = path_ff[-1]
            current_fix = path_fix[-1]

            # Construct grid of all stabilities of all amino acids at all sites
            AA_grid = np.array([_residues]*length)
            dG = np.zeros(AA_grid.shape, dtype=float)
            for (i,j), AA in np.ndenumerate(AA_grid):
                seq1 = mutant[:]
                seq1[i] = AA_grid[i,j]
                dG[i,j] = lattice.stability(seq1, target=target)

            dG = dG.round(decimals=9)
            # Calculate the fraction folded
            ff = fracfolded(dG)
            # normalized fixation probabilities
            fix = fixation(current_ff, ff)
            fix = fix / fix.sum()
            #print(fix)
            
            # Find where 95% of the probability density goes.
            x, y = kmax2d(fix, fix.size)
            prob = []
            for k in range(ff.size):
                prob.append(fix[y[k], x[k]])
                if sum(prob) > pdensity:
                    n_top_mutations = k - 1
                    break
                    
            # Walk out to the 95% density paths
            for k in range(n_top_mutations):
                
                # Check that the new moves are better than the current position
                newff = ff[y[k], x[k]]
                newfix = fix[y[k], x[k]]
                
                if newff < current_ff:
                    finished = False
                    new_move = mutant[:]
                    new_move[y[k]] = AA_grid[y[k], x[k]]
                    new_move = "".join(new_move)
                    
                    # Add paths
                    updated_unfinished_paths.append(path + [new_move])
                    updated_unfinished_ff.append(path_ff + [newff])
                    updated_unfinished_fix.append(path_fix + [newfix])
                    
                # If there is no fracfolded change, stay at the current position
                else:
                    finished_paths.append(path)
                    finished_ff.append(path_ff)
                    finished_fix.append(path_fix)
        m += 1
        unfinished_paths = updated_unfinished_paths
        unfinished_ff = updated_unfinished_ff
        unfinished_fix = updated_unfinished_fix
        print(m, len(unfinished_paths), n_top_mutations, )

    return finished_paths, finished_ff, finished_fix

In [4]:
from latticeproteins.sequences import random_sequence

In [5]:
seq1 = "DKCQCNWCRKFTDQR"
temp = 1 
length = len(seq1)
target = "UUURDRURDDLLDR"
# Build the conformation database
confs = Conformations(length, "database")

In [6]:
# Create a lattice protein calculator with given temperature and conf database.
lattice = LatticeThermodynamics(temp, confs)
#lattice_p = PredictedLattice(seq1, temp, confs, target=target)

In [38]:
#n_top_mutations = 9
#seq1 = "TGKGIHSICNQLMEL" #"".join(random_sequence(15))
a, stabsa = walk(seq1, lattice, pdensity=.05, target=target)
#p, stabsp = walk(seq1, lattice_p, n_top_mutations=n_top_mutations, target=target)

/Users/Zsailer/.virtualenvs/py3/lib/python3.5/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':
/Users/Zsailer/.virtualenvs/py3/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


1 7 11
2 78 20


KeyboardInterrupt: 

In [14]:
#n_top_mutations = 9
p, stabsp = walk(seq1, lattice_p, pdensity=.05, target=target)

/Users/Zsailer/.virtualenvs/py3/lib/python3.5/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':
/Users/Zsailer/.virtualenvs/py3/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


1 3 9
2 12 10
3 54 10
4 287 8
5 1603 30
6 8493 28


KeyboardInterrupt: 

In [20]:
import networkx as nx

# Construct networks


In [65]:
Ga = nx.DiGraph()
paths = np.unique(np.array(a))
for path in paths:
    for i, source in enumerate(path[:-1]):
        target_s = path[i+1]
        Ga.add_edge(source,target_s)
        
Gp = nx.DiGraph()
paths = np.unique(np.array(p))
for path in paths:
    for i, source in enumerate(path[:-1]):
        target_s = path[i+1]
        Gp.add_edge(source,target_s)
        

In [69]:
# Find all shared nodes.
anodes = set(Ga.nodes())
pnodes = set(Gp.nodes())
shared_nodes = anodes.intersection(pnodes)

# Finde shared edges.
aedges = set(Ga.edges())
pedges = set(Gp.edges())
shared_edges = aedges.intersection(pedges)
        
# ------------------------------------------
# Set shared edges
# ------------------------------------------
for s, t in Ga.edges():
    if (s,t) in shared_edges:
        Ga.edge[s][t]["shared"] = 1
    else:
        Ga.edge[s][t]["shared"] = 0

for s, t in Gp.edges():
    if (s,t) in shared_edges:
        Gp.edge[s][t]["shared"] = 1
    else:
        Gp.edge[s][t]["shared"] = 0
        
        
# ------------------------------------------
# Set shared nodes and their phenotype
# ------------------------------------------
for n in Ga.nodes():
    if n in shared_nodes:
        Ga.node[n]["shared"] = 1
    else:
        Ga.node[n]["shared"] = 0
    # Calculate dG
    Ga.node[n]["dG"] = lattice.stability(n, target=target) 
        
for n in Gp.nodes():
    if n in shared_nodes:
        Gp.node[n]["shared"] = 1
    else:
        Gp.node[n]["shared"] = 0
    Gp.node[n]["dG"] = lattice_p.stability(n, target=target) 


In [70]:
nx.write_gml(Gp, "/Users/Zsailer/Desktop/predicted.gml")
nx.write_gml(Ga, "/Users/Zsailer/Desktop/actual.gml")

In [2]:
def fracfolded(dG):
    return 1.0 / (1.0 + np.exp(dG))

In [30]:
def fixation(fitness1, fitness2, N=10e8, *args, **kwargs):
    """ Simple fixation probability between two organism with fitnesses 1 and 2.
    Note that N is the effective population size.
    .. math::
        p_{\\text{fixation}} = \\frac{1 - e^{-N \\frac{f_2-f_1}{f1}}}{1 - e^{-\\frac{f_2-f_1}{f1}}}
    """
    sij = (fitness2 - fitness1)/abs(fitness1)
    # Check the value of denominator
    denominator = 1 - np.exp(-N * sij)
    numerator = 1 - np.exp(- sij)
    # Calculate the fixation probability
    fixation = numerator / denominator
    print(sij)
    fixation = np.nan_to_num(fixation) 
    return fixation

In [31]:
x = np.random.uniform(size=(5,4))
fixation(.5, x)

[[ 0.60255954  0.23735457 -0.20503909 -0.41382138]
 [ 0.29101621 -0.99407846 -0.18426202 -0.68698845]
 [ 0.47680034  0.26821303  0.89402296  0.08421895]
 [ 0.39488356  0.67169487 -0.26096162  0.55545776]
 [-0.83965227  0.4764156   0.5008277  -0.72730853]]


/Users/Zsailer/.virtualenvs/py3/lib/python3.5/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


array([[ 0.45259127,  0.21128841,  0.        ,  0.        ],
       [ 0.25249644,  0.        ,  0.        ,  0.        ],
       [ 0.37923354,  0.23525515,  0.59099298,  0.08077003],
       [ 0.32624151,  0.48915797,  0.        ,  0.42619046],
       [ 0.        ,  0.37899466,  0.39397116,  0.        ]])

In [55]:
x = np.arange(5)
y = np.random.choice(5, size=5)
coor = np.column_stack((x,y))

In [56]:
coor

array([[0, 1],
       [1, 4],
       [2, 4],
       [3, 4],
       [4, 2]])

In [19]:
z = np.random.uniform(0,10, size=(5,5))

In [24]:
for c in coor:
    print(c, z[c])

(0, 3) 6.13686059416
(1, 0) 9.38058855215
(2, 3) 4.30063764448
(3, 1) 2.91769813651
(4, 1) 5.39408914596


In [58]:
z[coor]

array([[[ 0.58563509,  1.60956577,  6.58750551,  6.13686059,  4.22713002],
        [ 9.38058855,  8.63684649,  2.36109723,  3.72194388,  0.88740004]],

       [[ 9.38058855,  8.63684649,  2.36109723,  3.72194388,  0.88740004],
        [ 9.27166269,  5.39408915,  8.65816294,  0.85811775,  3.97609685]],

       [[ 4.28943712,  1.80100189,  5.72056801,  4.30063764,  4.02819849],
        [ 9.27166269,  5.39408915,  8.65816294,  0.85811775,  3.97609685]],

       [[ 6.22522665,  2.91769814,  4.23479357,  6.28037383,  2.93640258],
        [ 9.27166269,  5.39408915,  8.65816294,  0.85811775,  3.97609685]],

       [[ 9.27166269,  5.39408915,  8.65816294,  0.85811775,  3.97609685],
        [ 4.28943712,  1.80100189,  5.72056801,  4.30063764,  4.02819849]]])

In [62]:
z[x,y]

array([ 1.60956577,  0.88740004,  4.02819849,  2.93640258,  8.65816294])

In [75]:
def kmax(arr, k):
    """Return the (row, col) indices of the n largest arguments in the 2d array.

    Parameters
    ----------
    arr : array
        2d numpy array
    k : int
        number of arguments to select.

    Returns
    -------
    elements : 1-D array
        
    """
    if len(arr.shape) != 2:
        
    n, m = arr.shape
    vec = arr.flatten()
    vec_ = vec.argsort()[::-1]
    top_vec = vec_[:k]
    row = top_vec // n
    col = top_vec % n
    indices = np.column_stack((row, col))
    return arr[row, col], indices

In [76]:
x = np.random.uniform(0,10, size=(3,3))

In [80]:
stuff, coor = kmax(x, 2)
print(stuff)
print(coor)

[ 9.6750564   9.17476171]
[[0 2]
 [1 0]]


In [86]:
coor[:,1]

array([2, 0])

In [2]:
import numpy as np
from latticeproteins.sequences import _residues

In [4]:
AA_grid = np.array([_residues]*10)

In [6]:
AA_grid[0]

array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
       'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], 
      dtype='<U1')

In [67]:
import numpy as np
from latticeproteins.sequences import _residues
import np2d

def fixation(fitness1, fitness2, N=10e8, *args, **kwargs):
    """ Simple fixation probability between two organism with fitnesses 1 and 2.
    Note that N is the effective population size.
    .. math::
        p_{\\text{fixation}} = \\frac{1 - e^{-N \\frac{f_2-f_1}{f1}}}{1 - e^{-\\frac{f_2-f_1}{f1}}}
    """
    sij = (fitness2 - fitness1)/abs(fitness1)
    # Check the value of denominator
    denominator = 1 - np.exp(-N * sij)
    numerator = 1 - np.exp(- sij)
    # Calculate the fixation probability
    fixation = numerator / denominator
    fixation = np.nan_to_num(fixation) 
    return fixation

def mont_carlo_fixation_walk(seq, lattice, selected_trait="fracfolded", fix_threshold=0.5, max_mutations=15, target=None):
    """Use Monte Carlo method to walk
    
    Parameters
    ----------
    seq : str 
        seq
    lattice : LatticeThermodynamics object 
        Lattice protein calculator
    selected_trait : str
        The trait to select. 
    fix_threshold : float
        Fixation threshold.
    max_mutations : int (default = 15)
        Max number of mutations to make in the walk. 
    target : str 
        selected lattice target conformation. If None, the lattice will
        fold to the natural native conformation.
    """
    fitness_method = getattr(lattice, selected_trait)
    fitness0 = fitness_method(seq, target=target)
    finished = False
    mutant = list(seq[:])
    path, fitness, probs = (seq,), (fitness0,), (0,)
    # Monte Carlo move.
    m = 0
    while finished is False and m < max_mutations:

        # Construct grid of all stabilities of all amino acids at all sites
        AA_grid = np.array([_residues]*length)
        fits = np.zeros(AA_grid.shape, dtype=float)
        for (i,j), AA in np.ndenumerate(AA_grid):
            seq1 = mutant[:]
            seq1[i] = AA_grid[i,j]
            fits[i,j] = fitness_method(seq1, target=target)
            
            
        # Calculate fitness for all neighbors in sequence space
        fix = fixation(fitness0, fits)
        
        # Normalize
        denom = fix.sum()
        p = fix / denom
        
        # Sample moves
        mutation, indices = np2d.random.choice(AA_grid, p=p)
        site = indices[0,0]
        AA = indices[0,1]
        mutant[site] = mutation

        # Update our trajectory
        path += ("".join(mutant),)
        fitness += (fits[site, AA],)
        probs += (fix[site, AA],)
        fitness0 = fits[site, AA]
    
        # Check criteria to kill the trajectory
        # If the total probability of a mutation fixing is < 5%,
        # then kill the loop.
        if denom < fix_threshold:
            finished = True
        m += 1
        
    return path, fitness, probs

In [68]:
#seq1 = "DKCQCNWCRKFTDQR"
temp = 1 
length = len(seq1)
target = "UUURDRURDDLLDR"
# Build the conformation database
confs = Conformations(length, "database")
lattice = LatticeThermodynamics(temp, confs)

In [69]:
#for i in range(1000):
path, fitness, probs = mont_carlo_fixation_walk(seq1, lattice, target=target, max_mutations=40)

/Users/Zsailer/.virtualenvs/py3/lib/python3.5/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]
/Users/Zsailer/.virtualenvs/py3/lib/python3.5/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
